# Analyze composite jobs

This is a in-developing notebook enabling parseing QM results from opt / 
composite / frequence jobs. You can use this notebook to
- check job convergence
- view frequency modes
- check intermediate geometries

In [1]:
import os
import sys
# To add this RDMC into PYTHONPATH in case you haven't do it
sys.path.append(os.path.dirname(os.path.abspath('')))

import cclib  # This is not included in the environment and you may need to install manually
import numpy as np

from rdmc.view import mol_viewer, freq_viewer

from ipywidgets import interact, IntSlider, Dropdown

CRITERIA = {
    'gaussian': ['Force Maximum', 'Force RMS', 'Displacement Maximum', 'Displacement RMS'],
}

%load_ext autoreload
%autoreload 2

## 1. Load the file

In [2]:
log = "/Users/xiaorui/C3ddb server/Calcs/DMBENE/dmbene_25_cis/composite/input.log"
######################################
calc_results = cclib.io.ccread(log)
if not hasattr(calc_results, 'metadata'):
    raise RuntimeError('The cclib cannot path the file!')
else:
    package = calc_results.metadata['package'].lower()

## 2. Converged?

In [3]:
print(f'Optimization done?: {calc_results.optdone}')
if calc_results.optdone:
    
    last_converged_idx = [x for x, y in enumerate(calc_results.optstatus) \
                          if y & calc_results.OPT_DONE > 0][-1]
    xyz = cclib.io.ccwrite(calc_results,
                           outputtype='xyz',
                           indices=last_converged_idx,
                           returnstr=True,)
    
    viewer = mol_viewer(xyz, model='xyz')
    viewer.show()
    
    print('The optimized XYZ:\n')
    print(f'{xyz}')

Optimization done?: True


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The optimized XYZ:

13
[Geometry 36]
C     -0.4816190000   -0.5480710000    0.0181410000
O     -0.8880910000    0.6660510000   -0.3619790000
O      0.2074920000    2.0171080000    0.2559750000
H     -0.2261600000    2.6542300000   -0.3307180000
C      0.7186880000   -1.1403300000    0.1221180000
H      0.6969280000   -2.1918370000    0.3926190000
C     -1.8524250000   -0.9669740000    0.1119110000
H     -2.3107490000   -1.6127480000   -0.6301680000
H     -2.4859780000   -0.5066190000    0.8574650000
C      2.0459100000   -0.4838500000   -0.0897850000
H      2.6092480000   -1.0047780000   -0.8731580000
H      1.9230270000    0.5624040000   -0.3623780000
H      2.6551490000   -0.5305760000    0.8200550000



## 3. Find the geometry that is the closet to convergence

In [4]:
if not calc_results.optdone:
    
    criteria = CRITERIA[package]
    # Calculate the normalized target differences
    off_targets = (calc_results.geovalues - calc_results.geotargets) / calc_results.geotargets

    # set ones that meet target to 0s 
    off_targets[off_targets <= 0] = 0

    norm2targets = np.linalg.norm(off_targets, axis=1)
    sorted_conformers = np.argsort(norm2targets)

    best_conf = sorted_conformers[0]
    print(f'The best conformer is {best_conf+1}\n')
    print(f'Criteria:\n{"Item":<30}{"Value":<16}{"Threshold":<16}{"Off target":<16}')
    for item, val, target, off_target in zip(criteria,
                                 calc_results.geovalues[best_conf].tolist(),
                                 calc_results.geotargets.tolist(),
                                 off_targets[best_conf].tolist()):
        print(f'{item:<30}{val:<16}{target:<16}{off_target:<16.2e}')
    print("\n")
    xyz = cclib.io.ccwrite(calc_results,
                           outputtype='xyz',
                           indices=best_conf,
                           returnstr=True,)
    
    viewer = mol_viewer(xyz, model='xyz')
    viewer.show()
    
    print(f'{xyz}')
    

## 4. [IF A FREQ JOB] Visualize vibrations

In [5]:
try:
    num_neg_freq = np.sum(calc_results.vibfreqs < 0)
except AttributeError:
    num_neg_freq = None
    print('!!!!! THIS OUTPUT DOES NOT CONTAIN FREQUENCY INFORMATION !!!!')
    
print(f'!!!!! THIS JOB HAS {num_neg_freq} IMAGINARY FREQUENCIES !!!!!')
print(f'!!!!! PLEASE CHECK IF THIS IS A DESIRABLE BEHAVIOR !!!!!')

!!!!! THIS JOB HAS 1 IMAGINARY FREQUENCIES !!!!!
!!!!! PLEASE CHECK IF THIS IS A DESIRABLE BEHAVIOR !!!!!


In [6]:
def view_freqs(value):
    # Compose the xyz with viberation information
    freq_idx = np.where(calc_results.vibfreqs == value)[0][0]
#     print(freq_idx[0][0])
    lines = xyz.splitlines()
    vib_xyz_list = lines[0:2]
    for i, line in enumerate(lines[2:]):
        line = line.strip() + f'{"":12}'+ ''.join([f'{item:<12}' for item in calc_results.vibdisps[freq_idx][i].tolist()])
        vib_xyz_list.append(line)
    vib_xyz = '\n'.join(vib_xyz_list)
    
    viewer = freq_viewer(vib_xyz, model='xyz',)
    viewer.show()
    

if num_neg_freq is not None:
    dropdown = Dropdown(
        options=calc_results.vibfreqs,
        value=calc_results.vibfreqs[0],
        description='Vib freq:',
        disabled=False,
    )

interact(view_freqs, value=dropdown)

interactive(children=(Dropdown(description='Vib freq:', options=(-672.3256, 115.0627, 134.8722, 183.7553, 201.…

<function __main__.view_freqs(value)>

## 5. Visualize intermediate conformers

In [7]:
num_conf = calc_results.atomcoords.shape[0]
xyzs = [cclib.io.ccwrite(calc_results, outputtype='xyz',
                         indices=i, returnstr=True,)
        for i in range(num_conf)]

def view_intermediate_conformer(idx):
    xyz = xyzs[idx]
    mol_viewer(xyz, model='xyz').show()
    print(xyz)

interact(view_intermediate_conformer, idx=IntSlider(min=0, max=num_conf-1, step=1))

interactive(children=(IntSlider(value=0, description='idx', max=36), Output()), _dom_classes=('widget-interact…

<function __main__.view_intermediate_conformer(idx)>